In [2]:
!pip install transformers torch

In [3]:
from transformers import BertForSequenceClassification, BertTokenizer

In [4]:
import torch

In [5]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [21]:
def sentiment(tokens):

  output = model(**tokens)
  probs = torch.nn.functional.softmax(output[0], dim=-1)

  return probs

In [10]:
text = '''
Cathie Wood’s ARK Invest and asset manager 21Shares have applied for regulatory approval to offer a spot ethereum exchange-traded fund (ETF), per a filing with the U.S. Securities and Exchange Commission (SEC). If approved, this would be the first such product available in the U.S. and, as a spot ETF, it would directly invest in ETH.
Perspective: If approved, the “ARK 21Shares Ethereum ETF” could be a major institutional boon for ethereum and the crypto market at large, as it would provide an avenue for institutional investors to gain ethereum exposure without having to purchase or custody the crypto directly. As a spot ETF, it would also require the issuers to directly hold the asset themselves.
“This gives big entities, institutions, large family offices, investment firms, etc., etc., a chance to buy ethereum in a very easy way,” Aaron Arnold of Altcoin Daily explained in the video above. “They just buy a ticker, they don’t have to custody it.”
More details: The crypto market responded positively to the news of the filing, with the ether and bitcoin prices both seeing spikes.
    In its filing with the SEC, ARK and 21Shares indicated that the ETF would custody the assets with Coinbase Custody Trust Company, a custodian that has also been popular with recent filings for spot bitcoin ETFs.
    The filing comes as ARK and many others — including influential institutions like BlackRock and Fidelity — are awaiting responses for their requests to offer spot bitcoin ETFs.
    Cathie Wood’s ARK Invest and asset manager 21Shares have applied for regulatory approval to offer a spot ethereum exchange-traded fund (ETF), per a filing with the U.S. Securities and Exchange Commission (SEC). If approved, this would be the first such product available in the U.S. and, as a spot ETF, it would directly invest in ETH.
Perspective: If approved, the “ARK 21Shares Ethereum ETF” could be a major institutional boon for ethereum and the crypto market at large, as it would provide an avenue for institutional investors to gain ethereum exposure without having to purchase or custody the crypto directly. As a spot ETF, it would also require the issuers to directly hold the asset themselves.
“This gives big entities, institutions, large family offices, investment firms, etc., etc., a chance to buy ethereum in a very easy way,” Aaron Arnold of Altcoin Daily explained in the video above. “They just buy a ticker, they don’t have to custody it.”
More details: The crypto market responded positively to the news of the filing, with the ether and bitcoin prices both seeing spikes.
    In its filing with the SEC, ARK and 21Shares indicated that the ETF would custody the assets with Coinbase Custody Trust Company, a custodian that has also been popular with recent filings for spot bitcoin ETFs.
    The filing comes as ARK and many others — including influential institutions like BlackRock and Fidelity — are awaiting responses for their requests to offer spot bitcoin ETFs.
Cathie Wood’s ARK Invest and asset manager 21Shares have applied for regulatory approval to offer a spot ethereum exchange-traded fund (ETF), per a filing with the U.S. Securities and Exchange Commission (SEC). If approved, this would be the first such product available in the U.S. and, as a spot ETF, it would directly invest in ETH.
Perspective: If approved, the “ARK 21Shares Ethereum ETF” could be a major institutional boon for ethereum and the crypto market at large, as it would provide an avenue for institutional investors to gain ethereum exposure without having to purchase or custody the crypto directly. As a spot ETF, it would also require the issuers to directly hold the asset themselves.
“This gives big entities, institutions, large family offices, investment firms, etc., etc., a chance to buy ethereum in a very easy way,” Aaron Arnold of Altcoin Daily explained in the video above. “They just buy a ticker, they don’t have to custody it.”
More details: The crypto market responded positively to the news of the filing, with the ether and bitcoin prices both seeing spikes.
    In its filing with the SEC, ARK and 21Shares indicated that the ETF would custody the assets with Coinbase Custody Trust Company, a custodian that has also been popular with recent filings for spot bitcoin ETFs.
    The filing comes as ARK and many others — including influential institutions like BlackRock and Fidelity — are awaiting responses for their requests to offer spot bitcoin ETFs.
    Cathie Wood’s ARK Invest and asset manager 21Shares have applied for regulatory approval to offer a spot ethereum exchange-traded fund (ETF), per a filing with the U.S. Securities and Exchange Commission (SEC). If approved, this would be the first such product available in the U.S. and, as a spot ETF, it would directly invest in ETH.
Perspective: If approved, the “ARK 21Shares Ethereum ETF” could be a major institutional boon for ethereum and the crypto market at large, as it would provide an avenue for institutional investors to gain ethereum exposure without having to purchase or custody the crypto directly. As a spot ETF, it would also require the issuers to directly hold the asset themselves.
“This gives big entities, institutions, large family offices, investment firms, etc., etc., a chance to buy ethereum in a very easy way,” Aaron Arnold of Altcoin Daily explained in the video above. “They just buy a ticker, they don’t have to custody it.”
More details: The crypto market responded positively to the news of the filing, with the ether and bitcoin prices both seeing spikes.
    In its filing with the SEC, ARK and 21Shares indicated that the ETF would custody the assets with Coinbase Custody Trust Company, a custodian that has also been popular with recent filings for spot bitcoin ETFs.
    The filing comes as ARK and many others — including influential institutions like BlackRock and Fidelity — are awaiting responses for their requests to offer spot bitcoin ETFs.
'''


In [11]:
tokens = tokenizer.encode_plus(text, add_special_tokens=True)

In [12]:
len(tokens['input_ids'])

1350

In [13]:
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [24]:
start = 0
window_size = 510
tot_len = len(tokens['input_ids'])

loop = True
probs_list = []

while loop:
  end = start + window_size

  if end >= tot_len:
    loop=False
    end = tot_len

  # Extract window
  input_ids_chunk = [101] + input_ids[start:end] + [101]
  attention_mask_chunk = [1] + attention_mask[start:end] + [1]

  input_ids_chunk += [0] * (window_size - len(input_ids_chunk) +2)
  attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) +2)

  input_dict  ={
      'input_ids':torch.Tensor([input_ids_chunk]).long(),
      'attention_mask':torch.Tensor([attention_mask_chunk]).int(),
  }

  probs_list.append(sentiment(input_dict))
  start=end


In [25]:
probs_list[0]

tensor([[0.1574, 0.0138, 0.8288]], grad_fn=<SoftmaxBackward0>)

In [26]:
stack = torch.stack(probs_list)
stack

tensor([[[0.1574, 0.0138, 0.8288]],

        [[0.1076, 0.0289, 0.8635]],

        [[0.5649, 0.0165, 0.4186]]], grad_fn=<StackBackward0>)

In [27]:
stack.shape

torch.Size([3, 1, 3])

In [28]:
stack.reshape((3,3))

tensor([[0.1574, 0.0138, 0.8288],
        [0.1076, 0.0289, 0.8635],
        [0.5649, 0.0165, 0.4186]], grad_fn=<ReshapeAliasBackward0>)

In [29]:
stack.mean(1)

tensor([[0.1574, 0.0138, 0.8288],
        [0.1076, 0.0289, 0.8635],
        [0.5649, 0.0165, 0.4186]], grad_fn=<MeanBackward1>)

In [31]:
avg = stack.mean(dim=0)

In [32]:
avg

tensor([[0.2766, 0.0198, 0.7036]], grad_fn=<MeanBackward1>)

In [33]:
# Method for splitting tensors in torch

a = torch.arange(10)

torch.split(a, 4)

(tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8, 9]))

In [59]:
chunk_size = 512
tokens = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
input_id_chunks = list(tokens['input_ids'][0].split(chunk_size - 2))

In [61]:
for i in range(len(input_id_chunks)):
  input_id_chunks[i] = torch.cat([torch.Tensor([101]), input_id_chunks[i], torch.Tensor([102])])

  padding_len = chunk_size - len(input_id_chunks[i])

  if padding_len > 0:
    input_id_chunks[i] = torch.cat([input_id_chunks[i], torch.Tensor([0] * padding_len)])


In [62]:
input_id_chunks[2]

tensor([  101.,  1998., 11412.,  3208.,  2538.,  7377.,  6072.,  2031.,  4162.,
         2005., 10738.,  6226.,  2000.,  3749.,  1037.,  3962., 28855., 14820.,
         3863.,  1011.,  7007.,  4636.,  1006.,  3802.,  2546.,  1007.,  1010.,
         2566.,  1037., 15242.,  2007.,  1996.,  1057.,  1012.,  1055.,  1012.,
        12012.,  1998.,  3863.,  3222.,  1006., 10819.,  1007.,  1012.,  2065.,
         4844.,  1010.,  2023.,  2052.,  2022.,  1996.,  2034.,  2107.,  4031.,
         2800.,  1999.,  1996.,  1057.,  1012.,  1055.,  1012.,  1998.,  1010.,
         2004.,  1037.,  3962.,  3802.,  2546.,  1010.,  2009.,  2052.,  3495.,
        15697.,  1999.,  3802.,  2232.,  1012.,  7339.,  1024.,  2065.,  4844.,
         1010.,  1996.,  1523., 15745.,  2538.,  7377.,  6072., 28855., 14820.,
         3802.,  2546.,  1524.,  2071.,  2022.,  1037.,  2350., 12148., 23264.,
         2005., 28855., 14820.,  1998.,  1996., 19888.,  2080.,  3006.,  2012.,
         2312.,  1010.,  2004.,  2009., 